In [4]:

import kagglehub
import pandas as pd
import os

file_path = "microdados_enem_2016_coma.csv"

cols = [
    "NU_INSCRICAO", "SG_UF_RESIDENCIA",
    "NU_IDADE", "TP_SEXO", "TP_COR_RACA",
    "Q001", "Q002", "Q006", 
    "TP_ESCOLA", "TP_DEPENDENCIA_ADM_ESC",
    "NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_MT", "NU_NOTA_REDACAO",
    "NU_NOTA_HISTORIA" 
]

cols.extend([f"Q{str(i).zfill(3)}" for i in range(1, 51)])

chunks = []
chunksize = 100_000

for chunk in pd.read_csv(
    file_path,
    sep=",",
    encoding="latin1",
    on_bad_lines="skip",
    low_memory=False,
    chunksize=chunksize
):
    available_cols = [c for c in cols if c in chunk.columns]
    chunk = chunk[available_cols]
    chunks.append(chunk)
    print(f"Chunk lido: {len(chunk)} linhas")

df = pd.concat(chunks, ignore_index=True)

print("Dados carregados:", df.shape)
df = df.dropna(subset=["NU_NOTA_MT", "NU_NOTA_REDACAO", "NU_NOTA_CH"])
print("Shape pós-limpeza:", df.shape)

df.to_csv("enem2016_socioeconomico.csv", index=False, encoding="utf-8")

print("✅ Arquivo salvo: enem2016_socioeconomico.csv")


SyntaxError: invalid syntax (1882081581.py, line 9)

In [2]:
import kagglehub
import pandas as pd
import os


In [3]:
df = pd.read_csv(
    "enem2016_socioeconomico.csv",
    sep=",",          
    encoding="latin1", 
    on_bad_lines="skip", 
    low_memory=False   
)


# Limpeza: remove notas nulas
df = df.dropna(subset=["NU_NOTA_MT", "NU_NOTA_REDACAO", "NU_NOTA_CH"])
print("Shape pós-limpeza:", df.shape)
print(df.head())

   NU_INSCRICAO  NU_IDADE TP_SEXO  TP_COR_RACA Q001 Q002 Q006  TP_ESCOLA  \
0  160000301384      23.0       M            3    E    E    B          1   
1  160000000001      20.0       M            1    H    H    D          1   
2  160000000002      21.0       M            3    B    E    D          1   
3  160000000003      17.0       M            1    E    E    B          1   
4  160000000004      36.0       F            1    A    A    A          1   

   TP_DEPENDENCIA_ADM_ESC  NU_NOTA_CN  ...  Q041  Q042  Q043  Q044 Q045 Q046  \
0                     NaN       550.0  ...   NaN     C     A     A    A    A   
1                     NaN       576.6  ...   NaN     A     A     A    A    A   
2                     NaN       485.8  ...   NaN     B     A     B    A    A   
3                     NaN       571.4  ...   NaN     D     A     A    A    A   
4                     NaN         NaN  ...   NaN     A     A     A    A    A   

  Q047 Q048  Q049 Q050  
0    D    A     A    A  
1    D    A 